### 클러스터링

In [0]:
# 라이브러리 임포트
from kmeans_pytorch import kmeans
from pyspark.sql import Row, SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, PCA, StandardScaler
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager

font_dirs = ["/usr/share/fonts/truetype/nanum/"]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
 
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
 
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)
 
# pd.Series([-1,2,3]).plot(title='테스트', figsize=(3,2))
# pass

#### 1. 데이터 로드

In [0]:
# 데이터 로드
df_sample = spark.read.format("delta").table("database_pjt.3_use_encoding_sample")
print(f"데이터 크기: {df_sample.count()}개 행, {len(df_sample.columns)}개 컬럼")
display(df_sample.head(3))

Databricks data profile. Run in Databricks to view.

In [0]:
len(df_sample.columns)

#### 2. 피처 선택

In [0]:
# 수치형 컬럼만 추출 (클러스터링에 사용할 변수들)
from pyspark.sql.types import DoubleType, IntegerType, LongType, FloatType

numeric_cols = [
    f.name for f in df_sample.schema.fields
    if isinstance(f.dataType, (DoubleType, IntegerType, LongType, FloatType))
    and f.name not in ['row_id']  # ID 컬럼 제외
]

print(f"클러스터링에 사용할 수치형 컬럼 개수: {len(numeric_cols)}")
print("주요 컬럼들:", numeric_cols[:10])  # 처음 10개만 출력

# 결측값 확인
df_sample.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in numeric_cols[:5]]).show()

In [0]:
# 제외된 수치형 컬럼
excluded_numeric_cols = list(set(df_sample.columns) - set(numeric_cols))

print(f"❗ 제외된 수치형 컬럼 수: {len(excluded_numeric_cols)}")
print("제외된 컬럼 목록:", excluded_numeric_cols)

#### 3. 벡터화

In [0]:
# 모든 수치형 컬럼을 하나의 벡터로 결합
assembler = VectorAssembler(
    inputCols=numeric_cols,
    outputCol="features",
    handleInvalid="skip"  # 결측값이 있는 행은 제외
)

df_vectorized = assembler.transform(df_sample)
print("벡터화 완료!")

# 벡터 차원 확인
first_vector = df_vectorized.select("features").first()[0]
print(f"벡터 차원: {first_vector.size}")

#### 4. 스케일링

In [0]:
# 표준화 (평균 0, 분산 1로 조정)
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withStd=True,
    withMean=True
)

scaler_model = scaler.fit(df_vectorized)
df_scaled = scaler_model.transform(df_vectorized)

print("스케일링 완료!")
df_scaled.select("scaled_features").show(3, truncate=False)

In [0]:
df_scaled.count()

In [0]:
display(df_scaled)

Databricks data profile. Run in Databricks to view.

#### 5. 상관관계

# PCA 차원 축소

In [0]:
from pyspark.ml.feature import PCA

# PCA 모델을 k=6으로 설정하여 생성
pca = PCA(k=6, inputCol="scaled_features", outputCol="pcaFeatures")

# PCA 모델 학습 및 데이터 변환
pca_model = pca.fit(df_scaled)
pca_df = pca_model.transform(df_scaled)

print("PCA 적용 후 데이터 스키마:")
pca_df.printSchema()

# pcaFeatures 컬럼이 추가된 것을 확인할 수 있습니다.
pca_df.select("pcaFeatures").show(5, truncate=False)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import collections.abc  # To check for sequence type

# seaborn 스타일을 사용하여 그래프를 더 깔끔하게 만듭니다.
# plt.style.use('seaborn-v0_8-whitegrid')  # seaborn이 설치된 경우 활성화
# 한글 폰트가 깨질 경우를 대비한 설정 (환경에 맞게 설치 필요)
# import koreanize_matplotlib

def analyze_and_plot_pca_variance(pca_model, threshold=0.80):
    """
    PySpark 또는 Scikit-learn PCA 모델의 설명 분산을 분석하고 시각화합니다.

    Args:
        pca_model: 학습이 완료된 PySpark 또는 Scikit-learn PCA 모델 객체.
        threshold (float): 누적 설명력 기준값 (0.0 ~ 1.0).
    """
    # 1. 데이터 추출 및 검증
    explained_variance_ratio = None
    try:
        # PySpark 모델의 경우 .explainedVariance가 Vector 형태이므로 toArray()로 변환
        explained_variance_ratio = np.array(pca_model.explainedVariance.toArray())
    except AttributeError:
        # Scikit-learn 모델 호환성을 위함
        try:
            explained_variance_ratio = pca_model.explained_variance_ratio_
        except AttributeError:
            raise ValueError("입력된 객체에 'explainedVariance' 또는 'explained_variance_ratio_' 속성이 없습니다. 올바른 PCA 모델 객체인지 확인해주세요.")

    # [오류 방지 코드] 추출된 데이터가 배열/리스트 형태인지 확인
    if not isinstance(explained_variance_ratio, (np.ndarray, list)):
        raise ValueError(
            f"설명 분산 데이터가 배열 형태가 아닙니다 (현재 타입: {type(explained_variance_ratio)}). "
            f"PCA 모델이 올바르게 학습되었는지, 혹은 단일 주성분만 포함하고 있는지 확인해주세요."
        )

    cumulative_variance = np.cumsum(explained_variance_ratio)
    n_total_components = len(explained_variance_ratio)

    # 2. 목표 기준점을 넘는 지점 확인
    if np.any(cumulative_variance >= threshold):
        n_components_needed = np.argmax(cumulative_variance >= threshold) + 1
        actual_variance_at_threshold = cumulative_variance[n_components_needed - 1]
    else:
        n_components_needed = n_total_components
        actual_variance_at_threshold = cumulative_variance[-1]

    # 3. 결과 출력
    print(f"[✅ PCA 설명 분산 분석 결과]")
    print(f"총 {n_total_components}개의 주성분 중,")
    print(f"선택된 주성분 개수: {n_components_needed}개")
    print(f"이때의 누적 설명력: {actual_variance_at_threshold:.2%}")
    print(f"(목표 기준: {threshold:.0%} 이상)")

    # 4. 시각화
    fig, ax1 = plt.subplots(figsize=(12, 7))
    x_components = range(1, n_total_components + 1)
    
    ax1.bar(x_components, explained_variance_ratio, alpha=0.6, color='skyblue', label='개별 주성분 설명력')
    ax1.set_xlabel('주성분 개수', fontsize=12)
    ax1.set_ylabel('개별 주성분 설명력', fontsize=12, color='skyblue')
    ax1.tick_params(axis='y', labelcolor='skyblue')
    ax1.yaxis.set_major_formatter(PercentFormatter(1.0))

    ax2 = ax1.twinx()
    ax2.plot(x_components, cumulative_variance, 'o-', color='royalblue', label='누적 설명력')
    ax2.set_ylabel('누적 설명력', fontsize=12, color='royalblue')
    ax2.tick_params(axis='y', labelcolor='royalblue')
    ax2.yaxis.set_major_formatter(PercentFormatter(1.0))

    ax2.axhline(y=threshold, color='red', linestyle='--', label=f'목표 기준선 ({threshold:.0%})')
    ax2.axvline(x=n_components_needed, color='green', linestyle='--', label=f'필요 주성분 개수 ({n_components_needed}개)')
    ax2.plot(n_components_needed, actual_variance_at_threshold, 'ro', markersize=10, label=f'결과 지점 ({actual_variance_at_threshold:.2%})')

    plt.title('주성분 개수에 따른 설명 분산', fontsize=16, pad=20)
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='best')
    
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    fig.tight_layout()
    plt.show()

# --- 예제 사용법 ---
from unittest.mock import Mock
mock_pca_model = Mock()
# Scikit-learn 스타일
mock_pca_model.explained_variance_ratio_ = np.array(
     [0.25, 0.20, 0.15, 0.10, 0.08, 0.05, 0.04, 0.03, 0.02, 0.01] + [0.005]*10
)
# PySpark 스타일
mock_pca_model.explainedVariance.toArray.return_value = mock_pca_model.explained_variance_ratio_

# 함수 호출
analyze_and_plot_pca_variance(mock_pca_model, threshold=0.80)


In [0]:
pc_matrix = pca_model.pc.toArray()

In [0]:
pc1_loadings = pc_matrix[:, 0]

In [0]:
pc2_loadings = pc_matrix[:, 1]

In [0]:
# 추가
pc3_loadings = pc_matrix[:, 2] 

In [0]:
PC50_loading = pc_matrix[:, 5]

In [0]:
import pandas as pd

# 원본 특성 이름 리스트 (numeric_cols)가 이미 정의되어 있다고 가정합니다.
# 예시: numeric_cols = ['feature1', 'feature2', ..., 'feature196']

loadings_df = pd.DataFrame({
    'feature': numeric_cols,
    'PC1_loading': pc1_loadings,
    'PC2_loading': pc2_loadings,
    'PC3_loading': pc3_loadings,
    'PC50_loading': PC50_loading

    # 필요하다면 다른 주성분들도 추가할 수 있습니다 (예: PC3_loading = pc_matrix[:, 2])
})

# PC1에 대해 절대값이 큰 순서대로 정렬하여 상위 N개 특성 확인
print("--- PC1 Loadings (Top N) ---")
print(loadings_df.reindex(loadings_df.PC1_loading.abs().sort_values(ascending=False).index).head(10)) # 상위 10개

# PC2에 대해 절대값이 큰 순서대로 정렬하여 상위 N개 특성 확인
print("\n--- PC2 Loadings (Top N) ---")
print(loadings_df.reindex(loadings_df.PC2_loading.abs().sort_values(ascending=False).index).head(10)) # 상위 10개

In [0]:

# PC2에 대해 절대값이 큰 순서대로 정렬하여 상위 N개 특성 확인
print("\n--- PC50 Loadings (Top N) ---")
print(loadings_df.reindex(loadings_df.PC50_loading.abs().sort_values(ascending=False).index).head(10)) # 상위 10개

# 클러스터링 준비

In [0]:
# PCA 결과를 NumPy 배열로 변환
pca_vectors = pca_df.select("pcaFeatures").collect()
pca_array = np.array([row.pcaFeatures.toArray() for row in pca_vectors])

print(f"NumPy 배열 크기: {pca_array.shape}")

# GPU 텐서로 변환
pca_tensor = torch.from_numpy(pca_array).float()

# GPU 사용 가능하면 GPU로, 아니면 CPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pca_tensor = pca_tensor.to(device)

print(f"사용 디바이스: {device}")

#### 7. 최적의 클러스터 개수 찾기

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import numpy as np

# 테스트할 K값의 범위 설정
k_values = range(2, 12) # 2부터 11까지 테스트

# 결과를 저장할 리스트
wcss_scores = []
silhouette_scores = []

print("최적의 K를 찾기 위한 탐색을 시작합니다...")
for k in k_values:
    # K-Means 모델 생성 (중요: featuresCol을 'pcaFeatures'로 설정)
    kmeans = KMeans(featuresCol="pcaFeatures", k=k, seed=42)
    
    # 모델 학습
    model = kmeans.fit(pca_df)
    
    # 1. WCSS (오차제곱합) 계산
    wcss = model.summary.trainingCost
    wcss_scores.append(wcss)
    
    # 2. 실루엣 점수 계산
    predictions = model.transform(pca_df)
    evaluator = ClusteringEvaluator(featuresCol="pcaFeatures", metricName="silhouette", distanceMeasure="squaredEuclidean")
    silhouette = evaluator.evaluate(predictions)
    silhouette_scores.append(silhouette)
    
    print(f"K = {k} | WCSS = {wcss:.2f} | Silhouette Score = {silhouette:.4f}")

print("\n탐색 완료!")

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(k_values, wcss_scores, 'bo-')
plt.title('Elbow Method (엘보우 방법)')
plt.xlabel('클러스터 개수 (K)')
plt.ylabel('오차제곱합 (WCSS)')
plt.grid(True)
plt.show()

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, 'go-')
plt.title('Silhouette Scores (실루엣 점수)')
plt.xlabel('클러스터 개수 (K)')
plt.ylabel('실루엣 점수')
plt.grid(True)
plt.show()

#### 8. 최종 클러스터링

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.sql import functions as F

# 앞 단계에서 결정한 최적의 K값입니다. (실용적 관점에서 4를 사용)
best_k = 4 

print(f"최적 K={best_k}로 최종 클러스터링을 실행합니다...")

# 1. KMeans 모델 객체를 만듭니다. (아직 실행은 안 합니다)
#    - featuresCol: 클러스터링할 데이터가 담긴 컬럼명 (PCA 결과 컬럼)
#    - k: 클러스터 개수
#    - seed: 실행할 때마다 결과가 바뀌지 않도록 고정
final_kmeans = KMeans(featuresCol="pcaFeatures", k=best_k, seed=42)


# 2. .fit() 으로 모델을 학습시킵니다.
#    pca_df는 PCA 변환까지 완료된 데이터프레임입니다.
print("모델을 학습 중입니다...")
final_model = final_kmeans.fit(pca_df)


# 3. .transform() 으로 클러스터 ID('prediction' 컬럼)를 추가합니다.
#    이 한 줄로 기존 데이터에 클러스터링 결과가 자동으로 합쳐집니다.
print("각 데이터에 클러스터 ID를 할당 중입니다...")
df_final_result = final_model.transform(pca_df)


# 4. 최종 결과 확인
print("\n클러스터링 완료!")

# 이제 df_final_result에는 원본 컬럼 + 모든 변환 컬럼 + prediction 컬럼이 다 들어있습니다.
# 에러 없이 원하는 컬럼을 선택해서 볼 수 있습니다.
print("클러스터링 결과 샘플 확인 (상위 10개):")
df_final_result.select("기준년월", "prediction").show(10) # '기준년월' 등 원본 컬럼명을 넣어 확인해보세요.

# 클러스터별 데이터가 몇 개씩 있는지 확인
print("클러스터별 분포 확인:")
df_final_result.groupBy("prediction").count().orderBy("prediction").show()

In [0]:
import numpy as np

# --- 이 코드를 K-Means 실행 후, t-SNE 시각화 전에 추가하세요 ---

print("시각화를 위해 Spark 데이터프레임을 Pandas로 변환합니다...")

# 1. df_final_result에서 시각화에 필요한 두 컬럼('prediction', 'pcaFeatures')을 선택합니다.
# 2. .toPandas()를 사용해 Spark 데이터프레임을 Pandas 데이터프레임으로 변환합니다.
#    (데이터가 매우 클 경우 메모리 문제가 생길 수 있지만, 현재는 샘플링된 데이터라 괜찮습니다.)
pdf_for_viz = df_final_result.select("prediction", "pcaFeatures").toPandas()


# 3. 'prediction' 컬럼을 NumPy 배열로 변환합니다.
#    이것이 시각화 코드에서 사용할 'cluster_results'가 됩니다.
cluster_results = pdf_for_viz['prediction'].to_numpy()


# 4. 'pcaFeatures' 컬럼도 t-SNE가 사용할 수 있도록 NumPy 배열로 변환합니다.
pca_array = np.array(pdf_for_viz['pcaFeatures'].apply(lambda vec: vec.toArray()).tolist())


print("\n데이터 준비 완료!")
print(f" - pca_array 형태: {pca_array.shape}")
print(f" - cluster_results 형태: {cluster_results.shape}")
print(f" - cluster_results 고유값: {np.unique(cluster_results)}") # [0 1 2 3] 이 출력되는지 확인


In [0]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# t-SNE로 2D 축소
tsne_result = TSNE(n_components=2, perplexity=30, random_state=42).fit_transform(pca_array)

# Define colors
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

# 시각화
plt.figure(figsize=(10, 8))
for cluster_id in range(best_k):
    mask = cluster_results == cluster_id
    plt.scatter(tsne_result[mask, 0], tsne_result[mask, 1],
                c=colors[cluster_id % len(colors)],
                label=f'Cluster {cluster_id}', alpha=0.6)

plt.title('t-SNE 기반 클러스터 시각화')
plt.legend()
plt.grid(True)
plt.show()


In [0]:
import numpy as np

# best_k 변수와 cluster_results의 내용물을 직접 확인합니다.
print(f"시각화에 사용된 best_k: {best_k}")

unique_clusters, counts = np.unique(cluster_results, return_counts=True)

print("\n'cluster_results' 변수의 내용:")
for cluster_id, count in zip(unique_clusters, counts):
    print(f"  - 클러스터 {cluster_id}: {count} 개")

#### 9. 결과

In [0]:
# 클러스터별 데이터 분포
cluster_counts = df_final_result.groupBy("Prediction").count().orderBy("Prediction")
cluster_counts.show()

# 클러스터별 주요 특성 분석
agg_exprs = []
for col in numeric_cols[:10]:  # 주요 컬럼 10개만
    agg_exprs.append(F.mean(col).alias(f"avg_{col}"))
    agg_exprs.append(F.stddev(col).alias(f"std_{col}"))

cluster_profiles = df_final_result.groupBy("Prediction").agg(*agg_exprs)
cluster_profiles.show(truncate=False)

#### 10. 시각화

In [0]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA as SklearnPCA

# 2D 시각화를 위한 PCA
pca_2d = SklearnPCA(n_components=3)
pca_2d_result = pca_2d.fit_transform(pca_array)

# 클러스터별 색상으로 시각화
plt.figure(figsize=(10, 8))
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink']

for cluster_id in range(best_k):
    mask = cluster_results == cluster_id
    plt.scatter(
        pca_2d_result[mask, 0], 
        pca_2d_result[mask, 1], 
        c=colors[cluster_id % len(colors)], 
        label=f'Cluster {cluster_id}',
        alpha=0.6
    )

plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('소비정보 데이터 클러스터링 결과')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vector

# --- 이 코드를 사용하세요 ---

# 1. Vector에서 특정 인덱스의 값을 추출하는 Python 함수를 정의합니다.
def get_element(vector, index):
    try:
        # Vector 타입을 Python 리스트로 변환 후, 해당 인덱스의 값을 반환
        return float(vector.toArray()[index])
    except (IndexError, AttributeError):
        # 혹시 모를 오류 방지
        return None

# 2. 위 Python 함수를 Spark이 사용할 수 있는 UDF(사용자 정의 함수)로 등록합니다.
#    - 첫 번째 인자: 사용할 Python 함수
#    - 두 번째 인자: 이 함수가 반환하는 값의 데이터 타입 (실수형)
get_element_udf = F.udf(get_element, DoubleType())


# 3. UDF를 사용하여 'pc1'과 'pc2' 컬럼을 안전하게 생성합니다.
print("UDF를 사용하여 각 주성분(PC) 값을 추출합니다...")
df_with_components = df_final_result.withColumn("pc1", get_element_udf(F.col("pcaFeatures"), F.lit(0))) \
                                    .withColumn("pc2", get_element_udf(F.col("pcaFeatures"), F.lit(1)))

print("pc1, pc2 컬럼 생성 완료!")


# 4. 이제 pc1, pc2 컬럼이 정상적으로 생성되었으므로, 원래 하려던 집계를 실행합니다.
print("\n클러스터별 PCA 좌표 범위 계산:")
cluster_bounds = df_with_components.groupBy("cluster") \
    .agg(
        F.min("pc1").alias("pc1_min"),
        F.max("pc1").alias("pc1_max"),
        F.min("pc2").alias("pc2_min"),
        F.max("pc2").alias("pc2_max")
    ).orderBy("cluster")

cluster_bounds.show()


In [0]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # 3D 플롯을 위한 모듈
from sklearn.decomposition import PCA as SklearnPCA

# PCA로 3차원 축소
pca_3d = SklearnPCA(n_components=3)
pca_3d_result = pca_3d.fit_transform(pca_array)

# 클러스터별 색상 시각화
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink']

for cluster_id in range(best_k):
    mask = cluster_results == cluster_id
    ax.scatter(
        pca_3d_result[mask, 0], 
        pca_3d_result[mask, 1], 
        pca_3d_result[mask, 2], 
        c=colors[cluster_id % len(colors)],
        label=f'Cluster {cluster_id}',
        alpha=0.6
    )

ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')
ax.set_title('소비정보 데이터 클러스터링 결과 (3D)')
ax.legend()
plt.show()


In [0]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA as SklearnPCA

# 2D 시각화를 위한 PCA
pca_3d = SklearnPCA(n_components=3)
pca_3d_result = pca_3d.fit_transform(pca_array)

# 클러스터별 색상으로 시각화
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink']

for cluster_id in range(best_k):
    mask = cluster_results == cluster_id
    plt.scatter(
        pca_3d_result[mask, 0], 
        pca_3d_result[mask, 1], 
        pca_3d_result[mask, 2], 
        c=colors[cluster_id % len(colors)], 
        label=f'Cluster {cluster_id}',
        alpha=0.6
    )

ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('PCA 3')
plt.title('소비정보 데이터 클러스터링 결과')
plt.legend()
plt.grid(True)
plt.show()

### 저장

In [0]:
from pyspark.sql import functions as F

# df_sample은 가장 처음에 불러온 원본 데이터프레임입니다.
df_with_id = df_sample.withColumn("row_id", F.monotonically_increasing_id())

In [0]:
from pyspark.ml import Pipeline

# 지금까지 사용했던 각 단계의 모델 객체들을 순서대로 리스트에 담습니다.
# 각 변수명(assembler, scaler, pca, final_kmeans)은 실제 노트북에서 사용하신 이름으로 확인 후 맞춰주세요.
pipeline = Pipeline(stages=[
    assembler, 
    scaler, 
    pca, 
    final_kmeans
])

print("ML 파이프라인이 성공적으로 구성되었습니다.")

In [0]:
# 파이프라인 전체를 데이터에 학습(fit)시킵니다.
# df_with_id는 원본 데이터에 row_id가 추가된, 가장 초기 단계의 데이터프레임입니다.
pipeline_model = pipeline.fit(df_with_id)

# 학습된 파이프라인 모델을 원하는 경로에 저장합니다.
# 경로는 DBFS(Databricks File System) 경로를 사용하시면 됩니다.
# 여기서는 'customer_cluster_pipeline_model'이라는 이름으로 저장하겠습니다.
pipeline_path = "/FileStore/models/database_pjt/customer_cluster_pipeline_model"
pipeline_model.write().overwrite().save(pipeline_path)

print(f"학습된 파이프라인 모델이 여기에 저장되었습니다: {pipeline_path}")

In [0]:
# df_final_result는 클러스터링 결과가 포함된 최종 데이터프레임입니다.
# 원하는 테이블 이름을 지정합니다. (예: database_pjt.customer_clusters_final)
table_name = "database_pjt.customer_clusters_final"

# 데이터프레임을 Delta Table로 저장합니다.
# .mode("overwrite")는 기존에 같은 이름의 테이블이 있으면 덮어쓰는 옵션입니다.
df_final_result.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"클러스터링 최종 결과가 Delta Table로 저장되었습니다: {table_name}")